In [ ]:
%matplotlib inline     
import numpy as np
from matplotlib.pyplot import *
from scipy.sparse import diags
import matplotlib.pyplot as plt
from matplotlib import animation
import math
import random
from random import uniform
import pandas as pd
from scipy import stats
import pylab

## Echelle de temps

### Sur 1 an mois par mois

In [ ]:
temp_months = {"Algérie":[], "France":[]}

# Nous créons 2 listes vides dans lesquelles nous allons les remplir avec toutes les données du fichier .txt
    
    # cas de l'Algérie
    
temp_Alg = []

with open("Donnees/temperatures_algeria_1.txt","r") as algerie_1:
    line = algerie_1.readline()
    while line != "":
        data = float(line)
        temp_Alg.append(data)
        line = algerie_1.readline()

# cas de la France

temp_Fr = []

with open("Donnees/temperatures_france_1.txt","r") as france_1:
    
    line = france_1.readline()
    while line != "":
        data = float(line)
        temp_Fr.append(data)
        line = france_1.readline()

In [ ]:
def separate_months(L , country):
    """ list[float] * str -> list[float]
        on a en paramètres la liste de témpératures par heure sur un an. on veut retourner une liste de 12 éléments,
        c'est à dire une température moyenne par mois. """
    
    if country not in temp_months:
        print("Vous devez choisir entre l'Algérie et la France (attention à l'orthographe)")
        return
        
    else :
        
        sum_month = 0.0 
        hour = 1

        # on parcourt la liste (temp_pays_y). dès que l'on a atteint le nombre d'heures correspondant à 1 mois (730), 
        #on ajoute leur moyenne (average_temp) au dictionnaire associé au bon pays.

        for data in L:
            sum_month += data 
            if hour == 730:
                average_temp = sum_month/(24*30)
                temp_months[country].append(average_temp)
                average_temp = 0
                hour = 0
                sum_month = 0
            hour += 1

        return temp_months[country]

## Sur un an, jour par jour (donc 365 jours)

In [ ]:
temp_days = {"Algérie":[] , "France":[]}

def separate_days(L , country):
    """ list[float] * str -> list[float]
        on a en paramètres la liste de témpératures par heure sur un an. on veut retourner une liste de 365 éléments,
        c'est à dire une température moyenne par jour. """

    #on parcourt la liste. dès que l'on a atteint le nombre d'heures correspondant à 1 jour, on ajoute leur moyenne
    #au dictionnaire associé au bon pays
    
    if country not in temp_days:
        print("Vous devez choisir entre l'Algérie et la France (attention à l'orthographe)")
        return
    
    
    else : 
        
        sum_day = 0.0 
        hour = 1



        for data in L:
                sum_day += data 
                if hour == 24:
                    average_temp = sum_day/24
                    temp_days[country].append(average_temp)
                    average_temp = 0
                    hour = 0
                    sum_day = 0
                hour += 1

        return temp_days[country]

## Sur 24 heures (jour choisi par l'utilisateur)

In [ ]:
temp_hours = {"Algérie":[] , "France":[]}

def pick_a_day(L, country , month, day):
    ''' liste[float] * str * int * int -> list[float]
        on a en paramètres la liste de témpératures par heure sur un an. on veut retourner une liste de 24 éléments,
        c'est à dire la liste de départ tronquée.'''
    
    # vérifications de la validité des paramètres
    
    if (day < 1) or (day > 30):
        print("Le jour n'est pas dans le bon intervalle (il faut qu'il soit compris entre 1 et 30 inclus)\n")
        return 
    
    if (month < 1) or (month > 12):
        print("Le mois n'est pas dans le bon intervalle (il faut qu'il soit entre 1 et 12 inclus)\n")
        return
    
    if country not in temp_months:
        print("Vous devez choisir entre l'Algérie et la France (attention à l'orthographe)")
        return
    
    index_mois = 0
    index_jour = 0
    index = 0 
    liste = np.zeros(24)
    
    for i in range(1,8771):
        
        if (i%(30*24) == 0):
            index_mois = index_mois + 1
            
        if(index_mois == month):
            for j in range(i-719,i+1):
                
                liste[index] =  L[j-1]
                
                if(index+1 == 24):
                    index_jour = index_jour + 1
                    
                if(index_jour == day):
                    return liste
                
                elif(index+1 == 24):
                    index = 0
                    
                index = index + 1
    
    for j in range(i-719,i+1):
                
        liste[index] = L[j-1]
        
        if(index+1 == 24):
            index_jour = index_jour + 1

        if(index_jour == day):
            return liste

        elif(index+1 == 24):
            index = 0
            
        index = index + 1
     
            
    return liste

In [ ]:
temp_hours["France"] = pick_a_day(temp_Fr,"France",12,30).copy()

In [ ]:
temp_hours["Algérie"] = pick_a_day(temp_Fr,"Algérie",1,1).copy()

## Calcul des variables nécessaires pour implémentation numérique

In [2]:
def delta_t(L,echelle_temps):
    """ list[float] , int -> float
        on définit delta_t, la valeur de retour, en heure"""
    
    # cas mois par mois sur un an
    if(echelle_temps == 1):
        delta_t = 730
    
    # cas jour par jour sur un an 
    if(echelle == 2):
        delta_t = 24
        
    # cas heure par heure sur un jour
    if(echelle == 3):
        delta_t = 1
        
    else:
        print("Erreur dans le choix de l'échelle de temps")
        return 
    
    return delta_t
    

## Implémentation numérique : méthode implicite et explicite pour arriver à Crank Nicholson

In [1]:
# REMARQUE : éviter d'appeler une variable ou une fonction "lambda" car déjà prédéfini par python

# CE QUI SUIT EST UNE BASE POUR IMPLEMENTER LES FONCTIONS DES DEUX METHODES

# La fonction suivante permet de discrétiser un intervalle [0,1] en N+1 noeuds équidistants
# (c'est-à-dire on "tranche" notre colonne de terre en N+1 morceaux)

def intervalle_discret(N): # Pour commencer on peut prendre par exemple N=10
    x=np.linspace(0,1,N+1)
    return x



# La fonction suivante permet de calculer la constante alpha de l'équation différentielle 
# On détermine alpha grâce aux valeurs de la conductivité thermique du sol et à sa capacité thermique volumique
# ATTENTION: il faut bien utiliser les unités SI

def calcul_alpha(lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol):
    alpha=lambda_conductivite_thermique_sol/capacite_thermique_volumique_sol
    return alpha

# La fonction suivante permet de calculer la valeur de lambda qui servira dans le calcul des températures de la colonne au cours du temps

def calcul_lambda(alpha,delta_x,delta_t):
    lamb=alpha*delta_t/((delta_x)**2)
    return lamb

# La fonction suivante permet d'initialiser les températures de notre colonne de terre à l'instant t=0 dans un vecteur colonne de dimensions N-1 (et pas N attention)
# Il s'agit d'un choix arbitraire étant donné que les valeurs vont toujours finir par converger vers les températures à l'équilibre thermodynamique de la colonne de terre 
# On choisit ici d'initialiser notre colonne avec un température uniforme qui vaut 0
# ATTENTION: dans le programme qui étudiera la température sur plusieurs heures, il ne faudra pas réinitialiser à 0 à chaque nouvelle heure mais seulement au début de la première
def initialisation_T(N,T0):
    T=T0*np.ones((N-1,1)) # création d'un vecteur colonne avec que des zéros
    return T
    
# La fonction suivante permet de créer le vecteur qui sera utilisé pour calculer le vecteur T à l'étape k+1
# Il n'y a pas vraiment de nom pertinent pour cette fonction
# extremite fait référence aux températures aux extrémités de la colonne de terre (car elles interviennent uniquement dans ce vecteur)

def extremite(N,lamb,Tg,Td):
    ext=np.zeros((N-1,1))
    ext[0]=lamb*Tg
    ext[-1]=lamb*Td
    
    return ext


# On passe maintenant aux fonctions spécifiques à chaque méthode
# "e" correspond à explicite et "i" à implicite 

# METHODE EXPLICITE

# La fonction suivante permet de créer la matrice diagonale de taille (N-1)*(N-1)
# Cette matrice sera utilisée pour calculer le vecteur T à l'étape k+1 avec la méthode explicite
# ATENTION : la matrice diagonale est différente pour la méthode implicite

def matrice_diag_e(N,lamb):
    k = np.array([lamb*np.ones(N-2),1-2*lamb*np.ones(N-1),lamb*np.ones(N-2)])
    offset = [-1,0,1]
    A = diags(k,offset).toarray()
    return A

# La fonction suivante est l'implémentation de la méthode explicite
# Cette fonction reprend d'autres fonctions vues plus haut
# Elle retourne par ailleurs le vecteur colonne T au bout d'un certain temps donné par n*delta_t
# Chaque étape (soit chaque boucle) correspond à un avancement de delta_t dans le temps
# Plus on fait de boucles, plus on se rappoche de la valeur théorique des températures à l'équilibre thermodynamique dans la colonne
# En effet, on atteint à un certain moment des températures constantes dans la colonne de terre qar on a atteint l'équilibre
# Pour rappel on considère que l'on s'intéresse à une colonne de terre dont la température est initialement nulle et dont on applique une certaine température au niveau du sol et au niveau de la dernière couche de sol)
# Ainsi, pour plus de logique, on suppose que la température imposée sur la dernière couche du sol est égale à la température initiale de la colonne
# (cela semblerait étrange d'imposer une température dans le sol à la colonne de terre)


def methode_explicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n):
    
    x=intervalle_discret(N)
    alpha=calcul_alpha(lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol)
    lamb=calcul_lambda(alpha,delta_x,delta_t)
    
    T=initialisation_T(N,T0)  # ATTENTION: dans le programme qui étudiera la température sur plusieurs heures, il ne faudra pas réinitialiser à 0 à chaque nouvelle heure mais seulement au début de la première

    ext=extremite(N,lamb,Tg,Td)
    
    A=matrice_diag_e(N,lamb)
    
    
    for i in range(1,n+1):
        T=A.dot(T)+ext
        
    return T


# METHODE IMPLICITE

# La fonction suivante permet de créer la matrice diagonale de taille (N-1)*(N-1)
# Cette matrice sera utilisée pour calculer le vecteur T à l'étape k+1 avec la méthode implicite
# ATENTION : cette matrice diagonale est différente pour la méthode explicite

def matrice_diag_i(N,lamb):
    k = np.array([-lamb*np.ones(N-2),1+2*lamb*np.ones(N-1),-lamb*np.ones(N-2)])
    offset = [-1,0,1]
    A = diags(k,offset).toarray()
    return A

# La fonction suivante est l'implémentation de la méthode explicite
# Cette fonction reprend d'autres fonctions vues plus haut
# Elle retourne par ailleurs le vecteur colonne T au bout d'un certain temps donné par n*delta_t
# [Explications cf méthode explicite]

def methode_implicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n):
    
    x=intervalle_discret(N)
    alpha=calcul_alpha(lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol)
    lamb=calcul_lambda(alpha,delta_x,delta_t)
    
    T=initialisation_T(N,T0)   # ATTENTION: dans le programme qui étudiera la température sur plusieurs heures, il ne faudra pas réinitialiser à 0 à chaque nouvelle heure mais seulement au début de la première

    ext=extremite(N,lamb,Tg,Td)
    
    A=matrice_diag_i(N,lamb)
    
    for i in range(1,n+1):
        T= np.linalg.solve(A,T+ext)
    return T




In [2]:
Tg=10 
Td=0 
T0=0 

N=10

delta_x=0.1   
delta_t=10    

lambda_conductivite_thermique_sol=2.1
capacite_thermique_volumique_sol=1674*10**3

n=1000


In [3]:
# Méthode explicite

T_e=methode_explicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n)

# Affichage des températures de la colonne de terre après un temps de n*delta_t

print('T explicite =',T_e)


T explicite = [[5.24235033e+00]
 [2.12816148e+00]
 [6.86938897e-01]
 [1.81613622e-01]
 [4.03663426e-02]
 [7.70663722e-03]
 [1.28608985e-03]
 [1.90262788e-04]
 [2.49265682e-05]]


In [4]:
# Méthode implicite

T_i=methode_implicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n)

# Affichage des températures de la colonne de terre après un temps de n*delta_t

print('T implicite =',T_i)

T implicite = [[5.23980440e+00]
 [2.12695699e+00]
 [6.87200628e-01]
 [1.82111234e-01]
 [4.06384602e-02]
 [7.80323060e-03]
 [1.31210387e-03]
 [1.95952499e-04]
 [2.59555921e-05]]


In [5]:
# Méthode de Crank Nicolson

def crank_nicolson(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n):
    T_e=methode_explicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n)
    T_i=methode_implicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n)
    
    T=0.5*(T_e+T_i)
    
    return T

In [6]:
T=crank_nicolson(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n)
print('T=',T)

T= [[5.24107737e+00]
 [2.12755923e+00]
 [6.87069762e-01]
 [1.81862428e-01]
 [4.05024014e-02]
 [7.75493391e-03]
 [1.29909686e-03]
 [1.93107643e-04]
 [2.54410802e-05]]


In [7]:
#CASE TESTS

def crank_nicolson_bis(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n):
    T_e=methode_explicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n)
    T_i=methode_implicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n)
    
    T=0.5*delta_t*(T_e+T_i)
    
    return T

T=crank_nicolson_bis(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n)
print('T=',T)

T= [[5.24107737e+01]
 [2.12755923e+01]
 [6.87069762e+00]
 [1.81862428e+00]
 [4.05024014e-01]
 [7.75493391e-02]
 [1.29909686e-02]
 [1.93107643e-03]
 [2.54410802e-04]]
